In [ ]:
## Função para encontrar distribuição invariante
function dist_invariante(P;pk)
    nx = length(P[1,:])*length(pk[:,1])
    nz = length(P[1,:])
    Q = zeros(nx,nx)
    uns  = ones(nx,nx)
    um = ones(nx,1)
    Id1 = Matrix(I,nx,nx)
    for i in 1:nx
        for j in 1:nx
            a_i = div(j-1,nz) + 1 # índice de capital, para averiguar se é igrual à função política para aquele par
            b_i = (j - 1) % nz  + 1
            hoje_s = (i - 1)  % nz  + 1
            hoje_a= div(i-1,nz) + 1
            if a_i == pk[hoje_a,hoje_s]
                Q[i,j] = P[hoje_s,b_i]
            end
        end
    end
    λ = (Id1 - Q + uns)'\um
    return Q, λ
end

In [ ]:
# função de excesso de demanda de ativos
function hugget_capital_demand(dist_inva;grid_a)
    na = length(grid_a)
    nz = Int(length(dist_inva)/na)
    opa = reshape(dist_inva, nz,na)
    dist_a = sum(opa,dims = 1)
    dist_z = sum(opa,dims = 2)
    K = dot(grid_a,dist_a)
    return K, dist_a', dist_z, opa
end

In [ ]:
# função para taxa de juros market-clearing do mercado de ativos
function equilibrio_r(β,σ,ρ,γ)
    erro2 =2
    r_max = 1/β - 1
    r_min = 0
    matriz_M = Array
    Z, P = disc_t(9;mu= 0, sigma = σ, rho= ρ, m = 3)
    S = exp.(Z)
    grid_a = LinRange(-minimum(S)/r_max,maximum(S)/r_max,1000)
    r_0 = (r_max +r_min)/2
    while erro2 > 1e-5    
        v_ha, k_ha, c_ha = ha_mg_acelerado([50 100 200 1000];grid_z = S,P,β = β,γ = γ,r = r_0 )
        Q, λ =  dist_invariante(P; pk = k_ha)
        K_new, matriz_M = hugget_capital_demand(λ;grid_a)[vec([1 4])]
        r_new = r_0
        if K_new > 1e-5
            r_max = r_0
            r_min = r_min
            r_0 = (r_max +r_min)/2
            println("capital $K_new juros  $r_0")
            erro2 = abs(r_new - r_0)
        elseif K_new < -1e-5
            r_max = r_max
            r_min = r_0
            r_0 = (r_max + r_min)/2
            println("capital $K_new juros $r_0")
            erro2 = abs(r_new - r_0)
        else
            println("capital $K_new juros $r_0")
            erro2 = abs(r_new - r_0)
        end
    end
    return r_0, matriz_M, v_ha, k_ha, c_ha
end